**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

<font color='red' size="4">Import any additional libraries you need in the same code block that you use it.</font>

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## Question 1a - Review the relevant code samples provided on Canvas. Note: SimonX/Y indicates block size X and key size Y. The same applies to SpeckX/Y. Then, using Python crypto libraries in a secure way, implement the following:

In [1]:
#! pip install SimonSpeck
#! pip install pycryptodome
from Crypto.Cipher import AES
from Crypto.Hash import Poly1305
from simon import SimonCipher
from Crypto.Random import get_random_bytes
from Crypto.Util.number import bytes_to_long , long_to_bytes
import secrets

ModuleNotFoundError: No module named 'Crypto'

In [1]:
#! pip install SimonSpeck
#! pip install pycryptodome
from Cryptodome.Cipher import AES
from Cryptodome.Hash import Poly1305
from simon import SimonCipher
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.number import bytes_to_long , long_to_bytes
import secrets

### Q1a (a) [20 points] Implement EaM using Simon128/128-CTR as the cipher and Poly1305-AES as the MAC. Your implementation should support multiple plaintext blocks, and include decryption & mac verification support. 

In [2]:
# 6 points for tag generation/verification
def mac_poly1305(message: bytes, key: bytes) -> (bytes, bytes):
    """ 
    Securely generate the Poly1305-AES MAC Tag with the builtin Poly1305 library functions.
    Return the tag and secure nonce as bytes.
    """
    tag = b""
    nonce = b""
    
    # YOUR CODE HERE
    mac = Poly1305.new(key=key,cipher=AES) # initialise poly1305
    mac.update(message) # add message
    nonce=mac.nonce # get nonce
    tag=mac.digest() # get tag
    return (tag,nonce)

def mac_poly1305_verify(tag: bytes, message: bytes, key: bytes, nonce: bytes) -> bool:
    """ 
    Securely verify the Poly1305-AES MAC with the builtin Poly1305 library functions.
    Return True if MAC is valid, else return False. 
    Hint: Use try/except to catch any errors.
    """
    # YOUR CODE HERE
    mac = Poly1305.new(key=key, nonce=nonce, cipher=AES, data=message) 
    try:
        mac.verify(tag) # verify tag
        return True
    except:
        return False

In [3]:
ptxt = b"this  a random message that is sufficiently long"
key = b'UR=\x13\x00\x01=\x9cQ[[\xfc<F{`^\xb9\x0bS<\x956\x13\x7f-\xe1b\x95\x92\xee@'
tag_verify = b'\x18n\xc4\xb49k\r*~\xd5\x01\xe02\xce\xf6-'
nonce_verify = b'{\x1f\t:\x15?\x9b\xe4\x7f\xc0\x17T\xd6\x90`h'
verify = mac_poly1305_verify(tag_verify, ptxt, key, nonce_verify)
assert verify == True

In [4]:
ptxt = b"this  a random message that is sufficiently long"
key = b'UR=\x13\x00\x01=\x9cQ[[\xfc<F{`^\xb9\x0bS<\x956\x13\x7f-\xe1b\x95\x92\xee@'
tag , nonce = mac_poly1305(ptxt, key)
verify = mac_poly1305_verify(tag, ptxt, key, nonce)
assert verify == True
new_tag , new_nonce = mac_poly1305(ptxt, key)
assert new_nonce != nonce
assert new_tag != tag

In [5]:
ptxt = b"this  a random message that is sufficiently long"
key = b'UR=\x13\x00\x01=\x9cQ[[\xfc<F{`^\xb9\x0bS<\x956\x13\x7f-\xe1b\x95\x92\xee@'
wrong_nonce = b'\xa3*\xdf<\xf3v@{U\xb8\xb6z\xa3(&\xe3'
verify = mac_poly1305_verify(tag, ptxt, key, wrong_nonce)
assert verify == False

In [6]:
# 8 points for encryption/decryption

def encrypt_simon(message: bytes, key: bytes) -> (bytes, bytes):
    """ 
    Encrypt the message using Simon 128/128-CTR. Use the given key. 
    Return the ctxt and securely generated nonce as bytes.
    Hint: Use bytes_to_long(msg_block) to convert data to long int, 
    and long_to_bytes(big_int) to convert data back to bytes.
    Use the default counter argument for Simon Cipher.
    Simon Cipher should take nonce as an argument.
    """
    ctxt= b""
    nonce = b""
    
    # YOUR CODE HERE
    nonce=secrets.token_bytes(16) # generate nonce
    cipher=SimonCipher(bytes_to_long(key), key_size=128, block_size=128, mode='CTR',init=bytes_to_long(nonce)) # simon 128/128 CTR MODE
    block_size=16 #block size
    padded_ptxt=ptxt+b'\x00'*(block_size-len(ptxt)%block_size)#add zeros for padding
    for i in range(0,len(padded_ptxt),block_size):
        block=padded_ptxt[i:i+block_size] # get block
        ctxt+=long_to_bytes(cipher.encrypt(bytes_to_long(block))) # encrypt and add to ctxt
    return ctxt,nonce


def decrypt_simon(ctxt: bytes, key: bytes, nonce: bytes) -> bytes:
    """ 
    Decrypt the message using Simon 128/128-CTR. Use the given key and nonce. 
    Return the plaintext message.
    Hint: Use bytes_to_long(msg_block) to convert data to long int, 
    and long_to_bytes(big_int) to convert data back to bytes.
    Use the default counter argument for Simon Cipher.
    Simon Cipher should take nonce as an argument.
    """
    ptxt = b""
    # YOUR CODE HERE
    cipher=SimonCipher(bytes_to_long(key),key_size=128,block_size=128,mode='CTR',init=bytes_to_long(nonce)) # simon 128/128 CTR
    block_size=16 # block size
    for i in range(0,len(ctxt),block_size):
        block=ctxt[i:i+block_size] # get block
        decrypted_block=long_to_bytes(cipher.decrypt(bytes_to_long(block)),block_size) # decrypt block and add to ptxt
        ptxt+=decrypted_block
    return ptxt.rstrip(b'x\00') # remove zeros


In [7]:
nonce = b'\x03\xf9\x02Kp\xcc\x00\xc1\xe6\xea\xabo\xc9c9\xe9'
key = b'Bv\xc1.Xs\xa0\x0c\xabt\x8b\xe0Cs\xc0\x15'
ctxt = b'\xdb\xa2@\x91\x9e4\xd6Z\x8b_\x9e\x90Iz\x92`'

ptxt = decrypt_simon(ctxt, key, nonce)
assert ptxt == b"\x8c\xf2'1\x10\x89s\xbe;{\x18S\xac\x0e\x17\xd9"

In [8]:
ptxt = b"this  a random message that is sufficiently long"
key = b'Bv\xc1.Xs\xa0\x0c\xabt\x8b\xe0Cs\xc0\x15'
ctxt , nonce = encrypt_simon(ptxt, key)
new_ptxt = decrypt_simon(ctxt, key, nonce)
assert new_ptxt == ptxt
# test unique nonce
ctxt_new , nonce_new = encrypt_simon(ptxt, key)
assert nonce_new != nonce
assert ctxt_new != ctxt

In [9]:
# [4 points] Implement you own EaM_encrypt() function and implement an example to demonstrate its use.
# You can have a hardcoded message, but the remaining elements must be securely generated.
# Print the ciphertext and tag result in hex.


In [30]:
# YOUR CODE HERE
def EaM_encrypt(message: bytes, enc_key: bytes, mac_key: bytes) -> (bytes, bytes, bytes, bytes):
    ctxt, enc_nonce=encrypt_simon(message, enc_key) # encrypt ptxt
    tag, mac_nonce=mac_poly1305(message, mac_key) # get tag
    return ctxt, tag, enc_nonce, mac_nonce



In [31]:
ptxt=b"This is a sample message for Encrypt-and-MAC."
enc_key=get_random_bytes(16)  
mac_key=get_random_bytes(32)  

ctxt, tag, nonce_enc, nonce_mac=EaM_encrypt(ptxt, enc_key, mac_key)

print("Ciphertext :", ctxt.hex())
print("Tag :", tag.hex())

Ciphertext : cd846396eda52821526e5b65cdfd89b387d9958f42b872fdd7bca8650e824b0c21f044e9b0da2148785e1fb84b7576cc
Tag : 44955cf5089e16fd2aff5542847f57f6


In [32]:
# hidden test cases

In [33]:
#[2 points] Implement you own EaM_decrypt() function. 
#Demostrate its use by decrypting and verifying the EaM_encrypt result above. 
#Print the plaintext as a byte string and tag result.

In [34]:
# YOUR CODE HERE
def EaM_decrypt(ciphertext: bytes, tag: bytes, enc_nonce: bytes, mac_nonce: bytes, enc_key: bytes, mac_key: bytes) -> bytes:
    if not mac_poly1305_verify(tag, ciphertext, mac_key, mac_nonce):
        raise ValueError("MAC verification failed.") # verify tag
    ptxt = decrypt_simon(ciphertext, enc_key, enc_nonce) # decrypt ptxt 
    return ptxt


In [35]:
ptxt=b"This is a secret message"
enc_key=get_random_bytes(16)  # encryption key
mac_key=get_random_bytes(32)  # mac key
ctxt, tag, nonce_enc, nonce_mac=EaM_encrypt(ptxt, enc_key, mac_key) # encrypt and mac 
print("Ciphertext:", ctxt.hex())
print("Tag (hex):", tag.hex())
try:
    decrypted_ptxt=EaM_decrypt(ctxt, tag, nonce_enc, nonce_mac, enc_key, mac_key) # decryption of encrypt and mac
    print("Decrypted plaintext:", decrypted_ptxt)
except ValueError as e:
    print("Verification failed:", e)

Ciphertext (hex): 05dee926eaa884086f1478611af697ee9cb76aa05834e73ff150c1229bb3cc45
Tag (hex): e1a92a619a96568369912d4c45a9fa2b
Decrypted plaintext: b'This is a secret message'


In [75]:
# hidden test cases